In [1]:
import os
import pandas as pd
prot_list = pd.read_csv("protein_seqs.csv")['uniprot_id'].tolist()

In [2]:
import requests
from tqdm import tqdm
import json
folder_path = "prot_pdb"
os.makedirs(folder_path, exist_ok=True)
error_list = []

def af_download(prot, folder_path):
    url = f"https://alphafold.ebi.ac.uk/api/prediction/{prot}"
    request = requests.get(url)
    # print(json.dumps(request.json(), indent=2))
    pdb_url = request.json()[0]['pdbUrl']
    pdb_request = requests.get(pdb_url)
    pdb_content = pdb_request.text
    with open(os.path.join(folder_path, f"{prot}.pdb"), "w") as file:
        file.write(pdb_content)


In [48]:
print(len(prot_list), len(os.listdir(folder_path)))
print(len(error_list))

238 219
19


In [51]:
error_list = pd.DataFrame({"uniprot_id": error_list})
error_list.to_csv("prot_error_list.csv", index=False)

In [3]:
import json

def rcsb_download(prot, folder_path):
    url = f"https://www.ebi.ac.uk/proteins/api/proteins/{prot}"
    request = requests.get(url)
    data = request.json()

    pdb_refs = [ref for ref in data.get('dbReferences', []) if ref['type'] == 'PDB']

    if not pdb_refs:
        raise Exception(f"No PDB references found for {prot}")

    pdb_id = pdb_refs[0]['id']
    pdb_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"

    pdb_request = requests.get(pdb_url)
    if pdb_request.status_code != 200:
        raise Exception(f"Failed to download PDB file for {prot}")

    pdb_content = pdb_request.text
    with open(os.path.join(folder_path,f"{prot}.pdb"), "w") as file:
        file.write(pdb_content)


In [4]:
def swiss_download(prot, folder_path):
    url = f"https://swissmodel.expasy.org/repository/uniprot/{prot}.pdb"
    request = requests.get(url)
    if request.status_code != 200:
        raise Exception(f"Failed to download PDB file for {prot}")
    pdb_content = request.text
    with open(os.path.join(folder_path,f"{prot}.pdb"), "w") as file:
        file.write(pdb_content)

In [6]:
error_list = []
for prot in tqdm(prot_list):
    try:
        if os.path.exists(os.path.join(folder_path, f"{prot}.pdb")):
            continue
        # af_download(prot, folder_path)
        # rcsb_download(prot, folder_path)
        swiss_download(prot, folder_path)
    except Exception as e:
        error_list.append(prot)
        print(f"Error for {prot}: {e}")

100%|██████████| 238/238 [00:00<00:00, 57595.45it/s]
